In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map
from settings import *
from charts import stations_map

In [ ]:
STATION_COLORS = {
    'Buoy': '#e69800',
    'Synoptic': '#ff0000',
    'Rainfall': '#ffff00',
    'Climate': '#004da8',
    'WaveRide/SmartBayObsCenter': '#ffff00',
    'MI_Survey': '#ff0000',
    'EPA': '#002673'
}

MAP_LAYOUT = dict(
    legend=dict(title='<b>Station Type</b>',
                x=0.01),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=400,
    margin=dict(t=0, b=0, r=0, l=0),
    mapbox=dict(bearing=0,
                center=dict(
                    lat=53.4,
                    lon=352
                ),
                pitch=0,
                zoom=5,
                style="open-street-map"  # does not require token
                )
)

def stations_map(df):

    def stations_map_hovertemplate(df):
        return ['Name: '+'{}<br>'.format(n)+'
                'Type: '+'{}<br>'.format(t)+
                'Station No.: '+'{}<br>'.format(sN)+
                'County.: '+'{}<br>'.format(cnty)+
                'Open Year.: '+'{}<br>'.format(oY)+
                'Height: '+'{:.2f} m<br>'.format(h)+
                # 'Easting: '+'{}'.format(easting)+'<br>' +
                # 'Northing: '+'{}'.format(northing)+'<br>'
                'Lat: '+'{:.2f} \u00b0<br>'.format(lat)+
                'Lon: '+'{:.2f} \u00b0<br>'.format(lon)+
                '<extra></extra>'
                for n, t, sN, cnty, oY, cY, h, lat, lon in zip(list(df['name']),
                                                                            list(
                    df['Type']),
            list(
                    df['Station_Nu']),
            list(
                    df['County']),
            list(
                    df['Open_Year']),
            list(
                    df['Close_Year']),
            list(
                    df['Height__m_']),
            # list(
            #         df['Easting']),
            # list(
            #         df['Northing']),
            list(
                    df['Latitude']),
            list(
                    df['Longitude']),
        )]

    buoyDF = df.loc[df['Type'] == 'Buoy']
    synopticDF = df.loc[df['Type'] == 'Synoptic']
    rainfallDF = df.loc[df['Type'] == 'Rainfall']
    climateDF = df.loc[df['Type'] == 'Climate']
    tidbiTDF = df.loc[df['Type'] == 'TidbiT Sea Temperature Station']

    tidbiTTrend = go.Scattermapbox(
        name='tidbiT',
        lon=tidbiTDF.Longitude,
        lat=tidbiTDF.Latitude,
        marker=dict(color=STATION_COLORS['Buoy'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(tidbiTDF),
    )

    buoyTrend = go.Scattermapbox(
        name='Buoys',
        lon=buoyDF.Longitude,
        lat=buoyDF.Latitude,
        marker=dict(color=STATION_COLORS['Buoy'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(buoyDF),
    )
    rainfallTrend = go.Scattermapbox(
        name='Rainfall',
        lon=rainfallDF.Longitude,
        lat=rainfallDF.Latitude,
        marker=dict(color=STATION_COLORS['Rainfall'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(rainfallDF),
    )

    synopticTrend = go.Scattermapbox(
        name='Synoptic',
        lon=synopticDF.Longitude,
        lat=synopticDF.Latitude,
        marker=dict(color=STATION_COLORS['Synoptic'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(synopticDF),
    )

    climateTrend = go.Scattermapbox(
        name='Climate',
        lon=climateDF.Longitude,
        lat=climateDF.Latitude,
        marker=dict(color=STATION_COLORS['Climate'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(climateDF),
    )
    stations_map = go.Figure(
        data=[buoyTrend, synopticTrend, rainfallTrend, climateTrend,tidbiTTrend],
        layout=MAP_LAYOUT)
    
    return stations_map



In [ ]:
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.1/'
xls = pd.ExcelFile(
    data_path+'MeanAnnualSeaSurfaceTemperature_Anomalies_MalinHead.xlsx')
df = pd.read_excel(xls, 'MALIN_Timeseries')
movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 
df["5 Year Moving Average - Mean"]=movingAverageTotals
df

In [ ]:

annualTrace = go.Bar(x=df["year"],
                     y=df["Calculated Anomalies"],
                     text=df["temperature"],
                     name='Annual',
                     marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_2,
                            opacity=0.5
                            ),
                     hovertemplate='%{x}<br>' +
                            '<b>Annual</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["year"],
                     y=df["5 year moving average"],
                     text=df["5 Year Moving Average - Mean"],
                     name='5yr Moving Average',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                     hovertemplate='%{x}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
normal = go.Scatter(x=df["year"],
                     y=df["Unnamed: 5"],
                     name='1981-2010 Normal',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(color="#fdbf2d", #color used in report
                               width=1),
                     hoverinfo='skip',
                            )



In [ ]:
figure_3_1 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_1.add_trace(annualTrace,
            secondary_y=False,)
figure_3_1.add_trace(movingAverage,
            secondary_y=False,)
figure_3_1.add_trace(normal,
            secondary_y=True,)

figure_3_1.update_layout(TIMESERIES_LAYOUT)
figure_3_1.update_yaxes(title_text='Difference (°C) from 1981-2010 Normal',
                        secondary_y=False,
                        range=[-1, 1],
                        showgrid=False,
                        dtick=0.25,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_3_1.update_yaxes(title_text='Annual SST (°C)',
                        secondary_y=True,
                        range=[9.5, 11.6],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=10.6,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_3_1.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',  
    showspikes=True,  
    spikethickness=2, 
) 


In [ ]:
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.3/'
xls = pd.ExcelFile(
    data_path+'SubSurfaceTemperature_Anomalies_Rockall.xlsx')
df = xls.parse('Depth_Rockall', skiprows=20, index_col=None, na_values=['NA'])
# df = pd.read_excel(xls, 'Depth_Rockall')
# movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 
# df["5 Year Moving Average - Mean"]=movingAverageTotals
df

In [ ]:
annualTrace = go.Bar(x=df["Decimal Year"],
                     y=df["Temperature Anomaly °C"],
                     text=df["Temperature °C"],
                     name='Annual',
                     marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_2,
                            opacity=0.5
                            ),
                     hovertemplate='%{x}<br>' +
                            '<b>Annual</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["Decimal Year"],
                     y=df["5 year moving average"],
                    #  text=df["5 Year Moving Average - Mean"],
                     name='5yr Moving Average',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                     hovertemplate='%{x}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            # 'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
normal = go.Scatter(x=df["Decimal Year"],
                     y=df["1981-2010 (Normalized)"],
                     name='1981-2010 Normal',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(color="#fdbf2d", #color used in report
                               width=1),
                     hoverinfo='skip',
                            )


In [ ]:
figure_3_3 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_3.add_trace(annualTrace,
            secondary_y=False,)
figure_3_3.add_trace(movingAverage,
            secondary_y=False,)
figure_3_3.add_trace(normal,
            secondary_y=True,)

figure_3_3.update_layout(TIMESERIES_LAYOUT)
figure_3_3.update_yaxes(title_text='Difference (°C) from 1981-2010 Normal',
                        secondary_y=False,
                        range=[-0.25, 0.45],
                        showgrid=False,
                        dtick=0.05,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_3_3.update_yaxes(title_text='Annual SST (°C)',
                        secondary_y=True,
                        range=[3.93, 4.59],
                        showgrid=False,
                        dtick=0.1,  # dtick sets the distance between ticks
                        tick0=4.17,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_3_3.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',  
    showspikes=True,  
    spikethickness=2, 
) 


In [3]:
"""
Infrastructure Map
"""
DATA_PATH="/Users/dan/OneDrive - University College Cork/Status_Tool/"
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Map3.1/'
stationsDF = pd.read_csv(
            data_path+'Map3.1_StationTable_MI.txt')

rockallDF = pd.read_csv(
            data_path+'Map3.1_StationTable_RockallTroughSection.txt')
ellettLineDF = pd.read_csv(
            data_path+'Map3.1_StationTable_ExtendedEllettLineBuoy.txt')
tidbiDF = pd.read_csv(
            data_path+'Map3.1_StationTable_TidbiT.txt')
stationsDF 

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,0,Buoys,62091,M2_Buoy,0,0,0,53.483600,-5.430200,2001,,Buoy
1,1,Buoys,62092,M3_Buoy,0,0,0,51.215958,-10.548261,2002,,Buoy
2,2,Buoys,62093,M4_Buoy,0,0,0,54.666700,-9.066700,2007,,Buoy
3,3,Buoys,62094,M5_Buoy,0,0,0,51.690426,-6.704336,2004,,Buoy
4,4,Buoys,62095,M6_Buoy,0,0,0,53.074820,-15.881350,2006,,BuoySubSurf
5,5,Donegal,1575,Malin_head,20,241939,458562,55.372000,-7.339000,2009,,Synoptic
6,6,Mayo,0,Amets (Erris Peninsua),0,0,0,54.231766,-10.142453,0,,WaveRide
7,7,Galway,0,SmartBay Wave Buoy,0,0,0,53.228233,-9.268115,0,,WaveRide/SmartBayObsCenter
8,8,Cork,0,Ballycotton Harbour Tide Gauge,0,0,0,51.827800,-8.000700,0,,TideGauge


In [4]:
stationsDF_subsurface = stationsDF[stationsDF['name'].isin(['M6_Buoy', 'SmartBay Wave Buoy'])]
stationsDF_subsurface

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
4,4,Buoys,62095,M6_Buoy,0,0,0,53.074820,-15.881350,2006,,BuoySubSurf
7,7,Galway,0,SmartBay Wave Buoy,0,0,0,53.228233,-9.268115,0,,WaveRide/SmartBayObsCenter


In [5]:
rockallDF.columns

Index(['FID', 'County', 'Station_Nu', 'name', 'Height__m_', 'Easting',
       'Northing', 'Latitude', 'Longitude', 'Open_Year', 'Close_Year', 'Type'],
      dtype='object')

In [6]:
ellettLineDF

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,57.551807,-7.401944,NaN,NaN,ExtendedEllettLineBuoy
1,1,NaN,NaN,RockallTroughSection,NaN,NaN,NaN,57.500000,-11.000000,NaN,NaN,ExtendedEllettLineBuoy
2,2,NaN,NaN,NaN,NaN,NaN,NaN,57.485966,-12.221432,NaN,NaN,ExtendedEllettLineBuoy
3,3,NaN,NaN,NaN,NaN,NaN,NaN,57.621053,-13.527829,NaN,NaN,ExtendedEllettLineBuoy
4,4,NaN,NaN,NaN,NaN,NaN,NaN,57.647061,-13.744957,NaN,NaN,ExtendedEllettLineBuoy
5,5,NaN,NaN,NaN,NaN,NaN,NaN,57.850607,-14.224175,NaN,NaN,ExtendedEllettLineBuoy
6,6,NaN,NaN,NaN,NaN,NaN,NaN,58.233196,-15.308031,NaN,NaN,ExtendedEllettLineBuoy
7,7,NaN,NaN,NaN,NaN,NaN,NaN,58.644685,-16.520445,NaN,NaN,ExtendedEllettLineBuoy
8,8,NaN,NaN,NaN,NaN,NaN,NaN,59.025936,-17.324367,NaN,NaN,ExtendedEllettLineBuoy
9,9,NaN,NaN,NaN,NaN,NaN,NaN,59.454925,-18.289291,NaN,NaN,ExtendedEllettLineBuoy


In [7]:
rockallDF

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,0,NaN,0,RockallTroughSection,0,0,0,57.5,-11.0,0,0,ExtendedElletLineBuoy


In [8]:
tidbiDFNew=pd.DataFrame(columns=rockallDF.columns)
tidbiDFNew['name']=tidbiDF["localId"]
tidbiDFNew['Latitude']=tidbiDF["latitude"]
tidbiDFNew['Longitude']=tidbiDF["longitude"]
tidbiDFNew['Open_Year']=pd.to_datetime(tidbiDF["beginLifes"]).dt.year
tidbiDFNew['Close_Year']=pd.to_datetime(tidbiDF["endLifespa"]).dt.year
tidbiDFNew['Type']=tidbiDF["datasetNam"]
tidbiDFNew

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,NaN,NaN,NaN,Gearhies_Cuan Baoi,NaN,NaN,NaN,51.65131,-9.58638,2004,2016,TidbiT Sea Temperature Station
1,NaN,NaN,NaN,Palmers Point,NaN,NaN,NaN,51.64444,-9.80871,2004,2016,TidbiT Sea Temperature Station
2,NaN,NaN,NaN,Inishfanard,NaN,NaN,NaN,51.72552,-9.99875,2004,2016,TidbiT Sea Temperature Station
3,NaN,NaN,NaN,Red Flag,NaN,NaN,NaN,53.23706,-9.78851,2015,2016,TidbiT Sea Temperature Station
4,NaN,NaN,NaN,Ardmore,NaN,NaN,NaN,53.30030,-9.76548,2004,2016,TidbiT Sea Temperature Station
5,NaN,NaN,NaN,Oilean Iariath,NaN,NaN,NaN,53.28818,-9.73658,2004,2016,TidbiT Sea Temperature Station
6,NaN,NaN,NaN,Sea_Lax,NaN,NaN,NaN,53.37476,-9.86870,2005,2016,TidbiT Sea Temperature Station
7,NaN,NaN,NaN,Fraochoilean,NaN,NaN,NaN,53.57500,-10.03503,2004,2016,TidbiT Sea Temperature Station
8,NaN,NaN,NaN,Inishdeighil,NaN,NaN,NaN,53.60768,-9.90281,2004,2016,TidbiT Sea Temperature Station
9,NaN,NaN,NaN,Rosroe,NaN,NaN,NaN,53.62400,-9.86386,2004,2016,TidbiT Sea Temperature Station


In [9]:
combinedDF = pd.concat([stationsDF,ellettLineDF,tidbiDFNew])
combinedDF.reset_index
combinedDF

,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,0,Buoys,62091,M2_Buoy,0,0,0,53.483600,-5.430200,2001.0,,Buoy
1,1,Buoys,62092,M3_Buoy,0,0,0,51.215958,-10.548261,2002.0,,Buoy
2,2,Buoys,62093,M4_Buoy,0,0,0,54.666700,-9.066700,2007.0,,Buoy
3,3,Buoys,62094,M5_Buoy,0,0,0,51.690426,-6.704336,2004.0,,Buoy
4,4,Buoys,62095,M6_Buoy,0,0,0,53.074820,-15.881350,2006.0,,BuoySubSurf
...,...,...,...,...,...,...,...,...,...,...,...,...
33,NaN,NaN,NaN,Ocean Inver,NaN,NaN,NaN,54.610340,-8.353220,2015.0,2016,TidbiT Sea Temperature Station
34,NaN,NaN,NaN,Deenish,NaN,NaN,NaN,51.739550,-10.212340,2015.0,2016,TidbiT Sea Temperature Station
35,NaN,NaN,NaN,Creevin - Inver,NaN,NaN,NaN,54.629280,-8.290140,2004.0,2016,TidbiT Sea Temperature Station
36,NaN,NaN,NaN,Killary Inner_Mussel_Line,NaN,NaN,NaN,53.600000,-9.755280,2015.0,2016,TidbiT Sea Temperature Station


In [19]:
    map_3_1=stations_map(combinedDF)
    map_3_1.update_layout(
    mapbox=dict(bearing=0,
            center=dict(
                lat=56,
                lon=346
            ),
            pitch=0,
            zoom=3.8))
    map_3_1